In [1]:
import pandas as pd
import numpy as np
import funcs, gc, sqlite3

In [10]:
# bos ou rat
tipo = 'rat'

In [11]:
unclassif = funcs.get_unclassified_registers(tipo=tipo)

In [12]:
classificadores = funcs.get_df_classif()
unclassif.loc[:,'SETOR'] = unclassif.apply(lambda row: funcs.classifica_setor(row, classificadores), axis=1)
funcs.classifica_cia(unclassif)

unclassif

,RAT.NUM_ATIVIDADE,MUNICIPIO,LOGRADOURO,DES_ENDERECO,COMPLEMENTO_ENDERECO,NOME_BAIRRO,LOGRADOURO2,DES_ENDERECO2,SETOR,CIA
0,2019-061525316-001,DIVINOPOLIS,,SERINGUEIRAS,JARDIM DAS ACACIAS,,,,SAO JOSE,139 CIA
1,2020-005652932-001,DIVINOPOLIS,GERALDO TELES DE OLIVEIRA,GERALDO TELES DE OLIVEIRA,,NOVA SUICA,,,NITEROI,142 CIA
2,2020-006752607-001,DIVINOPOLIS,GERALDO TELES DE OLIVEIRA,GERALDO TELES DE OLIVEIRA,,NOVA SUICA,,,NITEROI,142 CIA
3,2020-012958766-001,DIVINOPOLIS,CINCO,CINCO,CASA,,,,NITEROI,142 CIA
4,2020-020217609-001,DIVINOPOLIS,TERRA E AREIA,TERRA E AREIA,,,,,BOM PASTOR,53 CIA
5,2020-022268150-001,DIVINOPOLIS,,JOSE DA COSTA GONTIJO,,,,,BOM PASTOR,53 CIA
6,2020-024510122-001,DIVINOPOLIS,,TULIPA,,,,,NITEROI,142 CIA
7,2020-024942192-001,DIVINOPOLIS,,PERNAMBUCO,,,,,PLANALTO,139 CIA
8,2020-025767344-001,DIVINOPOLIS,,CASTANHEIRA,JARDINOPOLIS,,,,NITEROI,142 CIA
9,2020-025466667-001,DIVINOPOLIS,,CRISTO REDENTOR,,,,,PLANALTO,139 CIA


In [13]:
new_classif_dict = [
    {
        'nrat': unclass_list[0],
        'setor': unclass_list[-2],
        'cia': unclass_list[-1]
    }
    for unclass_list in unclassif.values
]

new_classif_dict

[{'nrat': '2019-061525316-001', 'setor': 'SAO JOSE', 'cia': '139 CIA'},
 {'nrat': '2020-005652932-001', 'setor': 'NITEROI', 'cia': '142 CIA'},
 {'nrat': '2020-006752607-001', 'setor': 'NITEROI', 'cia': '142 CIA'},
 {'nrat': '2020-012958766-001', 'setor': 'NITEROI', 'cia': '142 CIA'},
 {'nrat': '2020-020217609-001', 'setor': 'BOM PASTOR', 'cia': '53 CIA'},
 {'nrat': '2020-022268150-001', 'setor': 'BOM PASTOR', 'cia': '53 CIA'},
 {'nrat': '2020-024510122-001', 'setor': 'NITEROI', 'cia': '142 CIA'},
 {'nrat': '2020-024942192-001', 'setor': 'PLANALTO', 'cia': '139 CIA'},
 {'nrat': '2020-025767344-001', 'setor': 'NITEROI', 'cia': '142 CIA'},
 {'nrat': '2020-025466667-001', 'setor': 'PLANALTO', 'cia': '139 CIA'},
 {'nrat': '2020-026232613-001', 'setor': 'NITEROI', 'cia': '142 CIA'},
 {'nrat': '2020-027085976-001', 'setor': 'NITEROI', 'cia': '142 CIA'},
 {'nrat': '2020-028955708-001', 'setor': 'PLANALTO', 'cia': '139 CIA'},
 {'nrat': '2020-029588829-001', 'setor': 'PLANALTO', 'cia': '139 CIA'

In [14]:
query_update = '''
UPDATE tbl_{}
    SET "SETOR" = ?,
    "CIA" == ?
WHERE
    "RAT.NUM_ATIVIDADE" = ?
'''.format(tipo)

with sqlite3.connect('gdo.db') as conn:
    cursor = conn.cursor()
    for dict_class in new_classif_dict:
        cursor.execute(query_update, (dict_class['setor'], dict_class['cia'], dict_class['nrat']))
        conn.commit()
conn.close()